## コード検索

このノートブックでは、Ada埋め込み（embeddings）を使ってセマンティックコード検索を実装する方法を示します。このデモンストレーションでは、独自の[openai-python コードリポジトリ](https://github.com/openai/openai-python)を使用しています。Pythonファイルから関数を抽出して埋め込み、インデックス化、クエリする簡易版のファイルパーシングを実装します。

### ヘルパー関数

まず、コードベースから重要な情報を抽出するための簡単なパーシング関数をセットアップします。

In [2]:
from typing import List, Dict, Generator
from pathlib import Path
import pandas as pd

DEF_PREFIXES = ['def ', 'async def ']
NEWLINE = '\n'

def extract_function_name(code: str) -> str:
    """
    'def'または'async def'で始まる行から関数名を抽出します。

    Args:
    - code (str): コード行

    Returns:
    - str: 関数名
    """
    for prefix in DEF_PREFIXES:
        if code.startswith(prefix):
            return code[len(prefix): code.index('(')]

def collect_until_out_of_scope(all_lines: List[str], start_idx: int) -> str:
    """
    関数定義が終わるまでのすべての行を収集します。

    Args:
    - all_lines (List[str]): 全コード行のリスト
    - start_idx (int): 関数定義の開始行のインデックス

    Returns:
    - str: 関数定義のコード
    """
    ret = [all_lines[start_idx]]
    for i in range(start_idx + 1, len(all_lines)):
        if all_lines[i].strip():
            ret.append(all_lines[i])
        else:
            break
    return NEWLINE.join(ret)

def parse_functions_in_file(filepath: str) -> Generator[Dict[str, str], None, None]:
    """
    Pythonファイル内のすべての関数を抽出します。

    Args:
    - filepath (str): Pythonファイルのパス

    Yields:
    - Dict[str, str]: 関数情報（コード、関数名、ファイルパス）
    """
    with open(filepath, 'r', encoding='utf-8') as file:
        all_lines = file.read().replace('\r', NEWLINE).split(NEWLINE)
        for idx, line in enumerate(all_lines):
            for prefix in DEF_PREFIXES:
                if line.startswith(prefix):
                    code = collect_until_out_of_scope(all_lines, idx)
                    function_name = extract_function_name(code)
                    yield {
                        'code': code,
                        'function_name': function_name,
                        'filepath': filepath,
                    }
                    break

def extract_all_functions_from_repo(code_root: Path) -> List[Dict[str, str]]:
    """
    リポジトリ内のすべての.py関数を抽出します。

    Args:
    - code_root (Path): コードのルートディレクトリのPathオブジェクト

    Returns:
    - List[Dict[str, str]]: すべての関数の情報
    """
    code_files = list(code_root.glob('**/*.py'))

    num_files = len(code_files)
    print(f'.pyファイルの総数: {num_files}')

    if num_files == 0:
        print('リポジトリが存在しないか、code_rootが正しく設定されていません。')
        return []

    all_functions = [
        func
        for code_file in code_files
        for func in parse_functions_in_file(str(code_file))
    ]

    num_functions = len(all_functions)
    print(f'抽出された関数の総数: {num_functions}')

    return all_functions


# データのロード

まず、`openai-python` フォルダを読み込み、上で定義した関数を用いて必要な情報を抽出します。

In [8]:
# Set user root directory to the 'openai-python' repository
root_dir = Path.home()
print(f'ユーザーのルートディレクトリ: {root_dir}')
# Assumes the 'openai-python' repository exists in the user's root directory
code_root = root_dir / 'openai-python'

# Extract all functions from the repository
all_funcs = extract_all_functions_from_repo(code_root)

ユーザーのルートディレクトリ: /root
.pyファイルの総数: 17
抽出された関数の総数: 45


コンテンツが用意できたので、このデータをテキスト埋め込み用の「text-embedding-ada-002」エンドポイントに渡して、ベクトル埋め込みを取得することができます。

In [9]:
from openai.embeddings_utils import get_embedding

df = pd.DataFrame(all_funcs)
df['code_embedding'] = df['code'].apply(lambda x: get_embedding(x, engine='text-embedding-ada-002'))
df['filepath'] = df['filepath'].map(lambda x: Path(x).relative_to(code_root))
df.to_csv("data/code_search_openai-python.csv", index=False)
df.head()

,code,function_name,filepath,code_embedding
0,def get_hyperlinks(url):\n # Try to open th...,get_hyperlinks,web-crawl-q-and-a/web-qa.py,"[-0.0062524136155843735, 0.012277018278837204,..."
1,"def get_domain_hyperlinks(local_domain, url):\...",get_domain_hyperlinks,web-crawl-q-and-a/web-qa.py,"[-0.008532723411917686, 0.019567523151636124, ..."
2,def crawl(url):\n # Parse the URL and get t...,crawl,web-crawl-q-and-a/web-qa.py,"[0.02908903919160366, 0.019636360928416252, 0...."
3,def remove_newlines(serie):\n serie = serie...,remove_newlines,web-crawl-q-and-a/web-qa.py,"[-0.00836277287453413, -0.0018864485900849104,..."
4,"def split_into_many(text, max_tokens=max_token...",split_into_many,web-crawl-q-and-a/web-qa.py,"[-0.006750369910150766, 0.016972357407212257, ..."


### テスト

エンドポイントをいくつかのシンプルなクエリでテストしてみましょう。もし`openai-python`リポジトリに慣れているなら、簡単な英語の説明だけで求める関数を簡単に見つけられることに気付くでしょう。

`search_functions`というメソッドを定義し、データベースに含まれるエンベディング、クエリ文字列、そしてその他の設定オプションを引数として取ります。データベースを検索するプロセスは以下のように動作します：

1. 最初にクエリ文字列（`code_query`）を`text-embedding-ada-002`でエンベディングします。ここでの理由は、'a function that reverses a string' というようなクエリ文字列と、'def reverse(string): return string[::-1]' のような関数が、エンベディングされたときに非常に類似しているでしょう。
2. 次に、クエリ文字列のエンベディングとデータベース内の全データポイントとのコサイン類似度を計算します。これによって各ポイントとクエリとの間の距離が得られます。
3. 最後に、クエリ文字列との距離に基づいて全データポイントをソートし、関数パラメータで要求された数の結果を返します。

In [10]:
from openai.embeddings_utils import cosine_similarity

def search_functions(df, code_query, n=3, pprint=True, n_lines=7):
    embedding = get_embedding(code_query, engine='text-embedding-ada-002')
    df['similarities'] = df.code_embedding.apply(lambda x: cosine_similarity(x, embedding))

    res = df.sort_values('similarities', ascending=False).head(n)

    if pprint:
        for r in res.iterrows():
            print(f"{r[1].filepath}:{r[1].function_name}  score={round(r[1].similarities, 3)}")
            print("\n".join(r[1].code.split("\n")[:n_lines]))
            print('-' * 70)

    return res

In [11]:
res = search_functions(df, 'fine-tuning input data validation logic', n=3)

chatbot-kickstarter/database.py:load_vectors  score=0.709
def load_vectors(client:Redis, input_list, vector_field_name):
    p = client.pipeline(transaction=False)
    for text in input_list:    
        #hash key
        key=f"{PREFIX}:{text['id']}"
----------------------------------------------------------------------
chatbot-kickstarter/database.py:create_hnsw_index   score=0.688
def create_hnsw_index (redis_conn,vector_field_name,vector_dimensions=1536, distance_metric='COSINE'):
    redis_conn.ft().create_index([
        VectorField(vector_field_name, "HNSW", {"TYPE": "FLOAT32", "DIM": vector_dimensions, "DISTANCE_METRIC": distance_metric}),
        TextField("filename"),
        TextField("text_chunk"),        
        NumericField("file_chunk_index")
    ])
----------------------------------------------------------------------
embeddings-playground/embeddings_playground.py:embedding_from_string  score=0.682
def embedding_from_string(input: str, model: str) -> list:
    response 

In [12]:
res = search_functions(df, 'find common suffix', n=2, n_lines=10)

chatbot-kickstarter/transformers.py:get_unique_id_for_file_chunk  score=0.714
def get_unique_id_for_file_chunk(filename, chunk_index):
    return str(filename+"-!"+str(chunk_index))
----------------------------------------------------------------------
file-q-and-a/nextjs-with-flask-server/server/utils.py:get_pinecone_id_for_file_chunk  score=0.709
def get_pinecone_id_for_file_chunk(session_id, filename, chunk_index):
    return str(session_id+"-!"+filename+"-!"+str(chunk_index))
----------------------------------------------------------------------


In [13]:
res = search_functions(df, 'Command line interface for fine-tuning', n=1, n_lines=20)

enterprise-knowledge-retrieval/assistant.py:initiate_agent  score=0.711
def initiate_agent(tools):
    prompt = CustomPromptTemplate(
        template=SYSTEM_PROMPT,
        tools=tools,
        # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
        # The history template includes "history" as an input variable so we can interpolate it into the prompt
        input_variables=["input", "intermediate_steps", "history"],
    )
----------------------------------------------------------------------
